## Asset-level impact calculations
Here 'asset-level' impacts means the impact of hazards on each asset in a portfolio, taken in isolation. This is, as opposed to portfolio-level impacts where the asset impacts are aggregated together – a topic for another notebook.  

In [ ]:
pip install nbformat pandas plotly requests

In [2]:
import requests
base_url = "https://physrisk-api2-sandbox.apps.odh-cl1.apps.os-climate.org/api/"

portfolio = {
    "items": [
        {
            "asset_class": "RealEstateAsset",
            "type": "Building/Industrial",
            "location": "Asia",
            "latitude": 24.0426,
            "longitude": 91.0158,
        },
        {
            "asset_class": "RealEstateAsset",
            "type": "Building/Industrial",
            "location": "Asia",
            "latitude": 22.6588,
            "longitude": 90.3373,
        }
    ]
}
request = {
    "assets": portfolio,
    "include_asset_level": True,
    "include_calc_details": True,
    "include_measures": True,
    "years": [2050],
    "scenario": "ssp585"
}

url = base_url + "get_asset_impact"
response = requests.post(url, json=request).json()

In [3]:
from typing import NamedTuple
asset0_impacts = response["asset_impacts"][1]["impacts"]

class Key(NamedTuple):
    hazard_type: str
    scenario_id: str
    year: str

asset0_impact_dict = {}
for i in asset0_impacts:
    key = i["key"]
    asset0_impact_dict[Key(key["hazard_type"], key["scenario_id"], key["year"])] = i

hazard_types = set(k.hazard_type for k in asset0_impact_dict.keys())
wind_impact_histo = asset0_impact_dict[Key("Wind", "historical", "None")]
wind_impact_ssp585 = asset0_impact_dict[Key("Wind", "ssp585", "2050")]

In [5]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

exceedance_histo = wind_impact_histo["impact_exceedance"]
exceedance_ssp585= wind_impact_ssp585["impact_exceedance"]

fig1 = make_subplots(rows=1, cols=2)
fig1.add_trace(go.Scatter(x=exceedance_histo['exceed_probabilities'], y=exceedance_histo["values"], name="baseline wind"), row=1, col=1)
fig1.add_trace(go.Scatter(x=exceedance_ssp585['exceed_probabilities'], y=exceedance_ssp585["values"], name="wind SSP585"), row=1, col=1)
fig1.update_xaxes(title="Annual exceedance probability", title_font={"size": 14}, row=1, col=1, type="log", autorange="reversed")
fig1.update_yaxes(title="Damage as fraction of insurable value", title_font={"size": 14}, row=1, col=1)
